# GAIL (Generative Adversarial Imitation Learning) 실습

이번 실습자료는 GAIL 저자 (Jonathan Ho)의  [구현코드](/https://github.com/openai/imitation.git)를 바탕으로 제작되었습니다. 

직접적인 구현 자체 (`TrainsitionClassifier`, `ImitationOptimizer`) 위주로 실습자료를 만들었고, 구현에 필요한 유틸리티 메서드들은 저자코드 저장소에서 그대로 받아와서 사용하도록 제작했습니다.

## 0. 환경설정

In [0]:
!rm -rf /usr/local/lib/python2.7/dist-packages/gym; rm -rf /usr/local/lib/python2.7/dist-packages/pyglet
!pip install virtualenv
!virtualenv gailenv
!apt-get install -y xvfb python-opengl swig ffmpeg zlib1g-dev python3-tk
!source /content/gailenv/bin/activate; pip install numpy==1.10.4 theano==0.8.2 gym==0.1.0 mujoco_py==0.4.0 JSAnimation PyOpenGL piglet pyglet==1.3.2 xlib pyvirtualdisplay box2d-py mako==1.0.7 

In [ ]:
import sys
sys.path.append('/content/gailenv/lib/python2.7/site-packages')
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()
import os
os.environ["DISPLAY"] = ":" + str(display.display) + "." + str(display.screen)

In [0]:
!git init .
!git remote add origin https://github.com/tzs930/imitation.git
!git pull origin master

## 1. Hyperparameter 설정


### 데이터 입력 관련

- `data` : 전문가 행동데이터(Expert trajectory)들이 저장된 `.h5` 파일의 경로를 지정합니다.
- `limit_trajs` : 최대 몇개의 Expert episode를 가져올지 지정합니다. 작아질수록 학습에 들어가는 데이터의 양이 적어집니다.
- `data_subsamp_freq` : 한 Episode 내에서 Subsampling을 얼마나 할지 지정합니다. 1일 경우 Subsampling을 하지 않고 모든 데이터를 사용합니다.

### Gym 환경 관련
- `env_name` : 어떤 Gym Environment에서 실험할 것인지 지정합니다. 반드시 `gym`에 등록(register)된 환경이어야 합니다. 
> e.g. `CartPole-v0`, `Acrobot-v0`, `MountainCar-v0`, ...
- `max_traj_len` :  Environment의 최대 에피소드 길이 값을 지정합니다.

In [0]:
# Expert dataset
data='expert_trajs/trajs_cartpole.h5'
limit_trajs=25
data_subsamp_freq=10

# MDP options
env_name='CartPole-v0'
max_traj_len=1000

### 학습 진행 관련 
- `min_total_sa` : 1번의 Policy Optimization에서 최대 몇개의 (s, a) 쌍을 샘플할 것인지 지정합니다.
- `max_iter` :  최대 몇 번의 Iteration을 학습할 것인지 지정합니다. 

한 번의 Iteration은 outer-loop에서 *Reward 학습*, inner-loop에서 *Policy 학습* 의 2단계로 이루어지며,  Inner loop에서 샘플된 (s,a)을 이용하여 TRPO를 학습하므로 사실상  `min_total_sa * max_iter` 만큼의 학습이 진행되게 됩니다.


### 저장 및 출력 관련 

- `print_freq` : 학습 도중 결과를 얼마나 자주 출력할지 지정합니다.
- `save_freq` :  학습 도중 결과 및 모델을 얼마나 자주 저장할지 지정합니다.
- `plot_freq` : 학습 도중 그래프를 얼마나 자주 출력할지 지정합니다.
- `logfilename` : 저장될 로그 및 학습결과의 경로를 지정합니다.

In [0]:
# Learning hyperparameters
min_total_sa=50000
max_iter=100

# Saving hyperparameters
print_freq=1
save_freq=10
plot_freq=10
logfilename='./' + env_name + '.h5'

### Reward 학습 관련 
- `reward_steps` : 한 번의 Reward optimization마다 몇 step의 update를 할 것인지 지정합니다.
- `reward_lr` :  Reward의 Learning Rate를 지정합니다.
- `reward_ent_reg_weight` :  Reward Loss에 적용되는 Entropy Regularization 정도를 지정합니다.
- `favor_zero_expert_reward` : Discriminator가 Expert trajectory를 1 또는 0으로 판단할 지 지정합니다. 가능한 한 빨리 도달해야하는 태스크들(ex. `MountainCar-v0`) 의 경우 이 옵션(`favor_zero_expert_reward=1`)이 필요합니다.


### TRPO (Trust-Region Policy Optimization) 관련 

GAIL은 inner-loop에서 RL Optimization을 이용하여 Policy를 학습시키는데, TRPO [(Schulman. J. et al., 2015)](https://arxiv.org/abs/1502.05477)를 사용하여 학습합니다. 

이번 실습에서는 TRPO가 중점이 아니므로 이 부분은 생략합니다.



In [0]:
# Reward hyperparameters
reward_steps=1
reward_lr=.01
reward_ent_reg_weight=.001
favor_zero_expert_reward=0

# TRPO hyperparameters
discount=.995
lam=.97
policy_max_kl=.01
policy_cg_damping=.1
vf_max_kl=.01
vf_cg_damping=.1
policy_ent_reg=0.

## 2. Generative Adversarial Learning을 통해 Reward를 학습 : `TrainsitionClassifier`

GAIL에서는 Discriminator를 local reward 혹은 cost function으로 삼아서 학습을 하게 됩니다. 논문에 따르면, Discriminator의 weight를 $w$, Discriminator network를 $D_w : \mathcal S \times \mathcal A  \to [0,1]$ 으로 정의하면, GAIL의 (local imaginary) reward function는 다음과 같습니다. 

&nbsp;

\begin{equation}
r(s,a) = \log D_w (s,a) 
\end{equation}
&nbsp;

다만 위 Reward function의 경우, 항상 0보다 작거나 같은 reward를 갖게 되므로, 다음과 같은 트릭을 사용하여 non-negative reward를 얻을 수 있습니다.

&nbsp;

### Positive (Non-Negative) Reward Function

\begin{equation}
r(s,a) = - \log(1- D_w (s,a) + \epsilon)
\end{equation}
&nbsp;

이 Reward function의 경우 $D_w$가 0이 되면 reward가 0, 1에 가까워질수록 점점 커지는 형태의 reward를 갖게 됩니다. 또한 $\infty$로 발산할 경우를 대비하여 $\epsilon$=1e-8 정도의 값을 $\log$ 안에 추가해주기도 합니다.

&nbsp;

### Negative (Non-Positive) Reward Function

앞서 말한 Reward function의 경우 non-negative reward를 학습하게 되는데, 태스크에 따라서 negative reward를 필요로 하게 되는 경우가 있습니다. 그 경우, 다음과 같이 reward function을 디자인할 수 있습니다.

\begin{equation}
r(s,a) = \log( D_w (s,a) + \epsilon )
\end{equation}
&nbsp;

이 Reward function의 경우 $D_w$가 0이 되면 reward가 $-\infty$, 1에 가까워질수록 0에 가까워주는 reward를 갖게 됩니다. 마찬가지로 $\infty$로 발산할 경우를 대비하여 $\epsilon$=1e-8 정도의 값을 $\log$ 안에 추가할 수 있습니다.

&nbsp;

### Learning a Discriminator 

논문에 따르면, Reward 학습을 위해 다음과 같은 Gradient를 이용해 Discriminator를 학습할 수 있습니다.

&nbsp;

$$\
\mathbb{E}_{\tau_i} [ \nabla_w log(D_w (s,a))] + \mathbb E_{\tau_E} [\nabla_w \log(1-D_w(s,a))] \tag{17}
$$

&nbsp;

이 코드에서는 위 Gradient를 이용해 학습하는 것과 같은 효과를 주기 위해, Expert trajectory를 0, Sampled trajectory를 1으로 labeling 한 후, Cross-Entropy Loss로 Regression을 하는 방법으로 Discriminator를 학습하고 있습니다.

&nbsp;



In [0]:
# from pyvirtualdisplay import Display
# display = Display(visible=0, size=(1400, 900))
# display.start()
# import os
# os.environ["DISPLAY"] = ":" + str(display.display) + "." + str(display.screen)

from policyopt import nn, rl, util, RaggedArray, ContinuousSpace, FiniteSpace, optim, thutil
import numpy as np
from contextlib import contextmanager
import theano; from theano import tensor

class TransitionClassifier(nn.Model):
    '''Reward/adversary for generative-adversarial training'''

    def __init__(self, obsfeat_space, action_space, hidden_spec, adam_lr, adam_steps, ent_reg_weight,
                 enable_inputnorm, time_scale, favor_zero_expert_reward, varscope_name):
        self.obsfeat_space, self.action_space = obsfeat_space, action_space
        self.hidden_spec = hidden_spec        
        self.adam_steps = adam_steps
        self.ent_reg_weight = ent_reg_weight; assert ent_reg_weight >= 0        
        self.time_scale = time_scale
        self.favor_zero_expert_reward = favor_zero_expert_reward

        with nn.variable_scope(varscope_name) as self.__varscope:
            # Map (s,a) pairs to classifier scores (log probabilities of classes)
            obsfeat_B_Df = tensor.matrix(name='obsfeat_B_Df')
            a_B_Da = tensor.matrix(name='a_B_Da', dtype=theano.config.floatX if self.action_space.storage_type == float else 'int64')
            t_B = tensor.vector(name='t_B')

            scaled_t_B = self.time_scale * t_B

            if isinstance(self.action_space, ContinuousSpace):
                # For a continuous action space, map observation-action pairs to a real number (reward)
                trans_B_Doa = tensor.concatenate([obsfeat_B_Df, a_B_Da], axis=1)
                trans_dim = self.obsfeat_space.dim + self.action_space.dim
                
                # Normalize
                with nn.variable_scope('inputnorm'):
                    self.inputnorm = (nn.Standardizer if enable_inputnorm else nn.NoOpStandardizer)(self.obsfeat_space.dim + self.action_space.dim)
                normedtrans_B_Doa = self.inputnorm.standardize_expr(trans_B_Doa)
                
                net_input = normedtrans_B_Doa
                net_input_dim = trans_dim
                
                # Compute scores
                with nn.variable_scope('hidden'):
                    net = nn.FeedforwardNet(net_input, (net_input_dim,), self.hidden_spec)
                with nn.variable_scope('out'):
                    out_layer = nn.AffineLayer(net.output, net.output_shape, (1,), initializer=np.zeros((net.output_shape[0], 1)))
                scores_B = out_layer.output[:,0]

            else:
                # For a finite action space, map observation observations to a vector of rewards

                # Normalize observations
                with nn.variable_scope('inputnorm'):
                    self.inputnorm = (nn.Standardizer if enable_inputnorm else nn.NoOpStandardizer)(self.obsfeat_space.dim)
                normedobs_B_Df = self.inputnorm.standardize_expr(obsfeat_B_Df)
                
                net_input = normedobs_B_Df
                net_input_dim = self.obsfeat_space.dim
                
                # Compute scores
                with nn.variable_scope('hidden'):
                    net = nn.FeedforwardNet(net_input, (net_input_dim,), self.hidden_spec)
                    
                with nn.variable_scope('out'):
                    out_layer = nn.AffineLayer(
                        net.output, net.output_shape, (self.action_space.size,),
                        initializer=np.zeros((net.output_shape[0], self.action_space.size)))
                scores_B = out_layer.output[tensor.arange(normedobs_B_Df.shape[0]), a_B_Da[:,0]]
        
        compute_scores_without_time = thutil.function([obsfeat_B_Df, a_B_Da], scores_B)
        self._compute_scores = lambda _obsfeat_B_Df, _a_B_Da, _t_B: compute_scores_without_time(_obsfeat_B_Df, _a_B_Da)

        # For preventing Numerical Error
        eps = 1e-8
        
        if self.favor_zero_expert_reward:
            # 0 for expert-like states, goes to -inf for non-expert-like states
            # compatible with envs with traj cutoffs for good (expert-like) behavior
            # e.g. mountain car, which gets cut off when the car reaches the destination
            
            ################# Implement Here #################
            
            raise NotImplementedError
            
            ##################################################
            
        else:
            # 0 for non-expert-like states, goes to +inf for expert-like states
            # compatible with envs with traj cutoffs for bad (non-expert-like) behavior
            # e.g. walking simulations that get cut off when the robot falls over
            rewards_B = -tensor.log(1. - tensor.nnet.sigmoid(scores_B) + eps)
            
        
        compute_reward_func = thutil.function([obsfeat_B_Df, a_B_Da], rewards_B)
        self._compute_reward = lambda _obsfeat_B_Df, _a_B_Da, _t_B: compute_reward_func(_obsfeat_B_Df, _a_B_Da)

        param_vars = self.get_trainable_variables()

        # Logistic regression loss, regularized by negative entropy
        labels_B = tensor.vector(name='labels_B')
        weights_B = tensor.vector(name='weights_B')
        losses_B = thutil.sigmoid_cross_entropy_with_logits(scores_B, labels_B)
        ent_B = thutil.logit_bernoulli_entropy(scores_B)
        loss = ((losses_B - self.ent_reg_weight*ent_B) * weights_B).sum(axis=0)
        lossgrad_P = thutil.flatgrad(loss, param_vars)

        adamstep_func = thutil.function(
            [obsfeat_B_Df, a_B_Da, labels_B, weights_B], loss,
            updates=thutil.adam(loss, param_vars, lr=adam_lr))
        
        self._adamstep = lambda _obsfeat_B_Df, _a_B_Da, _t_B, _labels_B, _weights_B: adamstep_func(_obsfeat_B_Df, _a_B_Da, _labels_B, _weights_B)

    @property
    def varscope(self): return self.__varscope

    def compute_reward(self, obsfeat_B_Df, a_B_Da, t_B):
        return self._compute_reward(obsfeat_B_Df, a_B_Da, t_B)

    def fit(self, obsfeat_B_Df, a_B_Da, t_B, exobs_Bex_Do, exa_Bex_Da, ext_Bex):
        # Transitions from the current policy go first, then transitions from the expert
        obsfeat_Ball_Df = np.concatenate([obsfeat_B_Df, exobs_Bex_Do])
        a_Ball_Da = np.concatenate([a_B_Da, exa_Bex_Da])
        t_Ball = np.concatenate([t_B, ext_Bex])

        # Update normalization
        self.update_inputnorm(obsfeat_Ball_Df, a_Ball_Da)

        B = obsfeat_B_Df.shape[0] # number of examples from the current policy
        Ball = obsfeat_Ball_Df.shape[0] # Ball - b = num examples from expert

        # Label expert as 1, current policy as 0
        labels_Ball = np.zeros(Ball)
        labels_Ball[B:] = 1.

        # Evenly weight the loss terms for the expert and the current policy
        weights_Ball = np.zeros(Ball)
        weights_Ball[:B] = 1./B
        weights_Ball[B:] = 1./(Ball - B); assert len(weights_Ball[B:]) == Ball-B

        # Optimize
        for _ in range(self.adam_steps):
            loss, kl, num_bt_steps = self._adamstep(obsfeat_Ball_Df, a_Ball_Da, t_Ball, labels_Ball, weights_Ball), None, 0

        # Evaluate
        scores_Ball = self._compute_scores(obsfeat_Ball_Df, a_Ball_Da, t_Ball); assert scores_Ball.shape == (Ball,)
        accuracy = .5 * (weights_Ball * ((scores_Ball < 0) == (labels_Ball == 0))).sum()
        accuracy_for_currpolicy = (scores_Ball[:B] <= 0).mean()
        accuracy_for_expert = (scores_Ball[B:] > 0).mean()
        assert np.allclose(accuracy, .5*(accuracy_for_currpolicy + accuracy_for_expert))

        return [
            ('rloss', loss, float), # reward function fitting loss
            ('racc', accuracy, float), # reward function accuracy
            ('raccpi', accuracy_for_currpolicy, float), # reward function accuracy
            ('raccex', accuracy_for_expert, float), # reward function accuracy
            ('rkl', kl, float),
            ('rbt', num_bt_steps, int),
            # ('rpnorm', util.maxnorm(self.get_params()), float),
            # ('snorm', util.maxnorm(scores_Ball), float),
        ]

    def update_inputnorm(self, obs_B_Do, a_B_Da):
        if isinstance(self.action_space, ContinuousSpace):
            self.inputnorm.update(np.concatenate([obs_B_Do, a_B_Da], axis=1))
        else:
            self.inputnorm.update(obs_B_Do)

    def plot(self, ax, idx1, idx2, range1, range2, n=100):
        assert len(range1) == len(range2) == 2 and idx1 != idx2
        x, y = np.mgrid[range1[0]:range1[1]:(n+0j), range2[0]:range2[1]:(n+0j)]

        if isinstance(self.action_space, ContinuousSpace):
            points_B_Doa = np.zeros((n*n, self.obsfeat_space.storage_size + self.action_space.storage_size))
            points_B_Doa[:,idx1] = x.ravel()
            points_B_Doa[:,idx2] = y.ravel()
            obsfeat_B_Df, a_B_Da = points_B_Doa[:,:self.obsfeat_space.storage_size], points_B_Doa[:,self.obsfeat_space.storage_size:]
            assert a_B_Da.shape[1] == self.action_space.storage_size
            t_B = np.zeros(a_B_Da.shape[0]) # XXX make customizable
            z = self.compute_reward(obsfeat_B_Df, a_B_Da, t_B).reshape(x.shape)
            
        else:
            obsfeat_B_Df = np.zeros((n*n, self.obsfeat_space.storage_size))
            obsfeat_B_Df[:,idx1] = x.ravel()
            obsfeat_B_Df[:,idx2] = y.ravel()
            a_B_Da = np.zeros((obsfeat_B_Df.shape[0], 1), dtype=np.int32) # XXX make customizable
            t_B = np.zeros(a_B_Da.shape[0]) # XXX make customizable
            z = self.compute_reward(obsfeat_B_Df, a_B_Da, t_B).reshape(x.shape)

        ax.pcolormesh(x, y, z, cmap='viridis')
        # ax.contour(x, y, z, levels=np.log(np.linspace(2., 3., 10)))
        ax.contourf(x, y, z, levels=[np.log(2.), np.log(2.)+.5], alpha=.5) # high-reward region is highlighted
        

## 3. 학습한 Reward로 Policy Optimization :  `ImitationOptimizer`

GAIL의 학습은 총 5단계로 이루어집니다.

1. 현재 Policy를 이용해 Trajectory를 Sampling 합니다.
2. 현재 Reward를 이용해 Baseline 혹은 Advantage function을 계산합니다.
3. 현재 Reward와 Sample된 Trajectory를 이용해 TRPO로 Policy를 학습합니다.
4. 주어진 Expert trajectory와 Sampled trajectory를 이용해 Reward를 학습합니다.
5. 학습된 Reward를 이용해 Value function (Q-function)을 학습합니다.

In [0]:
import pickle
class ImitationOptimizer(object):
    def __init__(self, mdp, discount, lam, policy, sim_cfg, step_func, reward_func, value_func,
                 policy_obsfeat_fn, reward_obsfeat_fn, policy_ent_reg, ex_obs, ex_a, ex_t):
        self.mdp, self.discount, self.lam, self.policy = mdp, discount, lam, policy
        self.sim_cfg = sim_cfg
        self.step_func = step_func
        self.reward_func = reward_func
        self.value_func = value_func
        # assert value_func is not None, 'not tested'
        self.policy_obsfeat_fn = policy_obsfeat_fn
        self.reward_obsfeat_fn = reward_obsfeat_fn
        self.policy_ent_reg = policy_ent_reg
        util.header('Policy entropy regularization: {}'.format(self.policy_ent_reg))

        assert ex_obs.ndim == ex_a.ndim == 2 and ex_t.ndim == 1 and ex_obs.shape[0] == ex_a.shape[0] == ex_t.shape[0]
        self.ex_pobsfeat, self.ex_robsfeat, self.ex_a, self.ex_t = policy_obsfeat_fn(ex_obs), reward_obsfeat_fn(ex_obs), ex_a, ex_t

        self.total_num_trajs = 0
        self.total_num_sa = 0
        self.total_time = 0.
        self.curr_iter = 0
        self.last_sampbatch = None # for outside access for debugging

    def step(self):
        with util.Timer() as t_all:

            # 1. Sample trajectories using current policy
            # print 'Sampling'
            with util.Timer() as t_sample:
                sampbatch = self.mdp.sim_mp(
                    policy_fn=lambda obsfeat_B_Df: self.policy.sample_actions(obsfeat_B_Df),
                    obsfeat_fn=self.policy_obsfeat_fn,
                    cfg=self.sim_cfg)
                samp_pobsfeat = sampbatch.obsfeat
                self.last_sampbatch = sampbatch

            # 2. Compute baseline / advantages
            # print 'Computing advantages'
            with util.Timer() as t_adv:
                # Compute observation features for reward input
                samp_robsfeat_stacked = self.reward_obsfeat_fn(sampbatch.obs.stacked)
                # Reward is computed wrt current reward function
                # TODO: normalize rewards
                rcurr_stacked = self.reward_func.compute_reward(samp_robsfeat_stacked, sampbatch.a.stacked, sampbatch.time.stacked)
                assert rcurr_stacked.shape == (samp_robsfeat_stacked.shape[0],)

                # If we're regularizing the policy, add negative log probabilities to the rewards
                # Intuitively, the policy gets a bonus for being less certain of its actions
                orig_rcurr_stacked = rcurr_stacked.copy()
                if self.policy_ent_reg is not None and self.policy_ent_reg != 0:
                    assert self.policy_ent_reg > 0
                    # XXX probably faster to compute this from sampbatch.adist instead
                    actionlogprobs_B = self.policy.compute_action_logprobs(samp_pobsfeat.stacked, sampbatch.a.stacked)
                    policyentbonus_B = -self.policy_ent_reg * actionlogprobs_B
                    rcurr_stacked += policyentbonus_B
                else:
                    policyentbonus_B = np.zeros_like(rcurr_stacked)

                rcurr = RaggedArray(rcurr_stacked, lengths=sampbatch.r.lengths)

                # Compute advantages using these rewards
                advantages, qvals, vfunc_r2, simplev_r2 = rl.compute_advantage(
                    rcurr, samp_pobsfeat, sampbatch.time, self.value_func, self.discount, self.lam)

            # 3. Learn a policy
            # print 'Fitting policy'
            with util.Timer() as t_step:
                params0_P = self.policy.get_params()
                step_print = self.step_func(
                    self.policy, params0_P,
                    samp_pobsfeat.stacked, sampbatch.a.stacked, sampbatch.adist.stacked,
                    advantages.stacked)
                self.policy.update_obsnorm(samp_pobsfeat.stacked)

            # 4. Fit reward function
            # print 'Fitting reward'
            with util.Timer() as t_r_fit:
                if True:#self.curr_iter % 20 == 0:
                    # Subsample expert transitions to the same sample count for the policy
                    inds = np.random.choice(self.ex_robsfeat.shape[0], size=samp_pobsfeat.stacked.shape[0])
                    exbatch_robsfeat = self.ex_robsfeat[inds,:]
                    exbatch_pobsfeat = self.ex_pobsfeat[inds,:] # only used for logging
                    exbatch_a = self.ex_a[inds,:]
                    exbatch_t = self.ex_t[inds]
                    rfit_print = self.reward_func.fit(samp_robsfeat_stacked, sampbatch.a.stacked, sampbatch.time.stacked, exbatch_robsfeat, exbatch_a, exbatch_t)
                else:
                    rfit_print = []

            # 5. Fit value function for next iteration
            # print 'Fitting value function'
            with util.Timer() as t_vf_fit:
                if self.value_func is not None:
                    # Recompute q vals # XXX: this is only necessary if fitting reward after policy
                    # qnew = qvals

                    # TODO: this should be a byproduct of reward fitting
                    rnew = RaggedArray(
                        self.reward_func.compute_reward(samp_robsfeat_stacked, sampbatch.a.stacked, sampbatch.time.stacked),
                        lengths=sampbatch.r.lengths)
                    qnew, _ = rl.compute_qvals(rnew, self.discount)
                    vfit_print = self.value_func.fit(samp_pobsfeat.stacked, sampbatch.time.stacked, qnew.stacked)
                else:
                    vfit_print = []

        # Log
        self.total_num_trajs += len(sampbatch)
        self.total_num_sa += sum(len(traj) for traj in sampbatch)
        self.total_time += t_all.dt
        fields = [
            ('iter', self.curr_iter, int),
            ('trueret', sampbatch.r.padded(fill=0.).sum(axis=1).mean(), float), # average return for this batch of trajectories
            ('iret', rcurr.padded(fill=0.).sum(axis=1).mean(), float), # average return on imitation reward
            ('avglen', int(np.mean([len(traj) for traj in sampbatch])), int), # average traj length
            ('ntrajs', self.total_num_trajs, int), # total number of trajs sampled over the course of training
            ('nsa', self.total_num_sa, int), # total number of state-action pairs sampled over the course of training
            ('ent', self.policy._compute_actiondist_entropy(sampbatch.adist.stacked).mean(), float), # entropy of action distributions
            ('vf_r2', vfunc_r2, float),
            ('tdvf_r2', simplev_r2, float),
            ('dx', util.maxnorm(params0_P - self.policy.get_params()), float), # max parameter difference from last iteration
        ] + step_print + vfit_print + rfit_print + [
            ('avgr', rcurr_stacked.mean(), float), # average regularized reward encountered
            ('avgunregr', orig_rcurr_stacked.mean(), float), # average unregularized reward
            ('avgpreg', policyentbonus_B.mean(), float), # average policy regularization
            # ('bcloss', -self.policy.compute_action_logprobs(exbatch_pobsfeat, exbatch_a).mean(), float), # negative log likelihood of expert actions
            # ('bcloss', np.square(self.policy.compute_actiondist_mean(exbatch_pobsfeat) - exbatch_a).sum(axis=1).mean(axis=0), float),
            ('tsamp', t_sample.dt, float), # time for sampling
            ('tadv', t_adv.dt + t_vf_fit.dt, float), # time for advantage computation
            ('tstep', t_step.dt, float), # time for step computation
            ('ttotal', self.total_time, float), # total time
        ]
        self.curr_iter += 1
        return fields

## 4. 메인함수 : GAIL Training 해보기

In [0]:
import argparse, h5py, json
import numpy as np
from environments import rlgymenv
import policyopt
from policyopt import imitation, nn, rl, util

# Policy architecture
OBSNORM_MODES = ('none', 'expertdata', 'online')
TINY_ARCHITECTURE = '[{"type": "fc", "n": 64}, {"type": "nonlin", "func": "tanh"}, {"type": "fc", "n": 64}, {"type": "nonlin", "func": "tanh"}]'
SIMPLE_ARCHITECTURE = '[{"type": "fc", "n": 100}, {"type": "nonlin", "func": "tanh"}, {"type": "fc", "n": 100}, {"type": "nonlin", "func": "tanh"}]'

obsnorm_mode='expertdata'
sim_batch_size=1
policy_hidden_spec=SIMPLE_ARCHITECTURE

### #1 : Environment & Policy 셋업
- Gym Environmnet 세팅
- Policy Architecture 구성
- Dataset Load

In [0]:
# main
mdp = rlgymenv.RLGymMDP(env_name)
util.header('MDP observation space, action space sizes: %d, %d\n' % (mdp.obs_space.dim, mdp.action_space.storage_size))

# Policy architecture
OBSNORM_MODES = ('none', 'expertdata', 'online')
TINY_ARCHITECTURE = '[{"type": "fc", "n": 64}, {"type": "nonlin", "func": "tanh"}, {"type": "fc", "n": 64}, {"type": "nonlin", "func": "tanh"}]'
SIMPLE_ARCHITECTURE = '[{"type": "fc", "n": 100}, {"type": "nonlin", "func": "tanh"}, {"type": "fc", "n": 100}, {"type": "nonlin", "func": "tanh"}]'

obsnorm_mode='expertdata'
sim_batch_size=1
policy_hidden_spec=SIMPLE_ARCHITECTURE

# Initialize the policy
enable_obsnorm = obsnorm_mode != 'none'
if isinstance(mdp.action_space, policyopt.ContinuousSpace):
    policy_cfg = rl.GaussianPolicyConfig(
        hidden_spec=policy_hidden_spec,
        min_stdev=0.,
        init_logstdev=0.,
        enable_obsnorm=enable_obsnorm)
    policy = rl.GaussianPolicy(policy_cfg, mdp.obs_space, mdp.action_space, 'GaussianPolicy')
else:
    policy_cfg = rl.GibbsPolicyConfig(
        hidden_spec=policy_hidden_spec,
        enable_obsnorm=enable_obsnorm)
    policy = rl.GibbsPolicy(policy_cfg, mdp.obs_space, mdp.action_space, 'GibbsPolicy')

util.header('Policy architecture')
for v in policy.get_trainable_variables():
    util.header('- %s (%d parameters)' % (v.name, v.get_value().size))
util.header('Total: %d parameters' % (policy.get_num_params(),))

Gym version: 0.1.0
MDP observation space, action space sizes: 4, 1



/usr/local/lib/python2.7/dist-packages/gym/envs/registration.py:13: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


Loading feedforward net specification
[
  {
    "type": "fc",
    "n": 100
  },
  {
    "type": "nonlin",
    "func": "tanh"
  },
  {
    "type": "fc",
    "n": 100
  },
  {
    "type": "nonlin",
    "func": "tanh"
  }
]
Affine(in=4, out=100)
Nonlinearity(func=tanh)
Affine(in=100, out=100)
Nonlinearity(func=tanh)
Affine(in=100, out=2)


WARNING (theano.tensor.blas): We did not found a dynamic library into the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


Policy architecture
- /GibbsPolicy/hidden/FeedforwardNet/layer_0/AffineLayer/W (400 parameters)
- /GibbsPolicy/hidden/FeedforwardNet/layer_0/AffineLayer/b (100 parameters)
- /GibbsPolicy/hidden/FeedforwardNet/layer_2/AffineLayer/W (10000 parameters)
- /GibbsPolicy/hidden/FeedforwardNet/layer_2/AffineLayer/b (100 parameters)
- /GibbsPolicy/out/AffineLayer/W (200 parameters)
- /GibbsPolicy/out/AffineLayer/b (2 parameters)
Total: 10802 parameters


### #2 : Expert Trajectory 불러오기

In [0]:
# Load expert data
def load_dataset(filename, limit_trajs, data_subsamp_freq):
    # Load expert data
    with h5py.File(filename, 'r') as f:
        # Read data as written by vis_mj.py
        full_dset_size = f['obs_B_T_Do'].shape[0] # full dataset size
        dset_size = min(full_dset_size, limit_trajs) if limit_trajs is not None else full_dset_size

        exobs_B_T_Do = f['obs_B_T_Do'][:dset_size,...][...]
        exa_B_T_Da = f['a_B_T_Da'][:dset_size,...][...]
        exr_B_T = f['r_B_T'][:dset_size,...][...]
        exlen_B = f['len_B'][:dset_size,...][...]

    print 'Expert dataset size: {} transitions ({} trajectories)'.format(exlen_B.sum(), len(exlen_B))
    print 'Expert average return:', exr_B_T.sum(axis=1).mean()
 
    # Stack everything together
    start_times_B = np.random.RandomState(0).randint(0, data_subsamp_freq, size=exlen_B.shape[0])
    print 'start times'
    print start_times_B
    exobs_Bstacked_Do = np.concatenate(
        [exobs_B_T_Do[i,start_times_B[i]:l:data_subsamp_freq,:] for i, l in enumerate(exlen_B)],
        axis=0)
    exa_Bstacked_Da = np.concatenate(
        [exa_B_T_Da[i,start_times_B[i]:l:data_subsamp_freq,:] for i, l in enumerate(exlen_B)],
        axis=0)
    ext_Bstacked = np.concatenate(
        [np.arange(start_times_B[i], l, step=data_subsamp_freq) for i, l in enumerate(exlen_B)]).astype(float)

    assert exobs_Bstacked_Do.shape[0] == exa_Bstacked_Da.shape[0] == ext_Bstacked.shape[0]# == np.ceil(exlen_B.astype(float)/data_subsamp_freq).astype(int).sum() > 0

    print 'Subsampled data every {} timestep(s)'.format(data_subsamp_freq)
    print 'Final dataset size: {} transitions (average {} per traj)'.format(exobs_Bstacked_Do.shape[0], float(exobs_Bstacked_Do.shape[0])/dset_size)

    return exobs_Bstacked_Do, exa_Bstacked_Da, ext_Bstacked
  
exobs_Bstacked_Do, exa_Bstacked_Da, ext_Bstacked = load_dataset(
    data, limit_trajs, data_subsamp_freq)

assert exobs_Bstacked_Do.shape[1] == mdp.obs_space.storage_size
assert exa_Bstacked_Da.shape[1] == mdp.action_space.storage_size
assert ext_Bstacked.ndim == 1

# Start optimization
max_traj_len = max_traj_len if max_traj_len is not None else mdp.env_spec.timestep_limit
print 'Max traj len:', max_traj_len

Expert dataset size: 3400 transitions (17 trajectories)
Expert average return: 200.0
start times
[4 0 3 3 3 1 3 2 4 0 0 4 2 1 0 1 1]
Subsampled data every 5 timestep(s)
Final dataset size: 680 transitions (average 40.0 per traj)
Max traj len: 1000


### #3 : Reward, Value Function, Optimizer 정의하기

In [0]:
reward = TransitionClassifier(
    hidden_spec=policy_hidden_spec,
    obsfeat_space=mdp.obs_space,
    action_space=mdp.action_space,    
    adam_lr=reward_lr,
    adam_steps=reward_steps,
    ent_reg_weight=reward_ent_reg_weight,
    enable_inputnorm=True,    
    time_scale=1./mdp.env_spec.timestep_limit,
    favor_zero_expert_reward=bool(favor_zero_expert_reward),
    varscope_name='TransitionClassifier')

vf = rl.ValueFunc(
    hidden_spec=policy_hidden_spec,
    obsfeat_space=mdp.obs_space,
    enable_obsnorm=obsnorm_mode != 'none',
    enable_vnorm=True,
    max_kl=vf_max_kl,
    damping=vf_cg_damping,
    time_scale=1./mdp.env_spec.timestep_limit,
    varscope_name='ValueFunc')

opt = ImitationOptimizer(
    mdp=mdp,
    discount=discount,
    lam=lam,
    policy=policy,
    sim_cfg=policyopt.SimConfig(
        min_num_trajs=-1, min_total_sa=min_total_sa,
        batch_size=sim_batch_size, max_traj_len=max_traj_len),
    step_func=rl.TRPO(max_kl=policy_max_kl, damping=policy_cg_damping),
    reward_func=reward,
    value_func=vf,
    policy_obsfeat_fn=lambda obs: obs,
    reward_obsfeat_fn=lambda obs: obs,
    policy_ent_reg=policy_ent_reg,
    ex_obs=exobs_Bstacked_Do,
    ex_a=exa_Bstacked_Da,
    ex_t=ext_Bstacked)

# Set observation normalization
if obsnorm_mode == 'expertdata':
    policy.update_obsnorm(exobs_Bstacked_Do)
    if reward is not None: reward.update_inputnorm(opt.reward_obsfeat_fn(exobs_Bstacked_Do), exa_Bstacked_Da)
    if vf is not None: vf.update_obsnorm(opt.policy_obsfeat_fn(exobs_Bstacked_Do))

Loading feedforward net specification
[
  {
    "type": "fc",
    "n": 100
  },
  {
    "type": "nonlin",
    "func": "tanh"
  },
  {
    "type": "fc",
    "n": 100
  },
  {
    "type": "nonlin",
    "func": "tanh"
  }
]
Affine(in=4, out=100)
Nonlinearity(func=tanh)
Affine(in=100, out=100)
Nonlinearity(func=tanh)
Affine(in=100, out=2)


NotImplementedError: ignored

### #4 : Training

`logfilename`에 해당하는 파일이 이미 존재하면, 아래 코드는 실행되지 않습니다. 반복해서 트레이닝을 하고 싶을 경우 파일을 지우거나,  `logfilename`을 바꾸면 됩니다.

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline
log = nn.TrainingLog( logfilename, [] )
for i in xrange(max_iter):
    iter_info = opt.step()
    log.write(iter_info, print_header=i % (20*print_freq) == 0, display=i % print_freq == 0)
    if save_freq != 0 and i % save_freq == 0 and log is not None:
        log.write_snapshot(policy, i)

    if plot_freq != 0 and i % plot_freq == 0:
        # Observation from expert
        exdata_N_Doa = exobs_Bstacked_Do # np.concatenate([exobs_Bstacked_Do, exa_Bstacked_Da], axis=1)
        # Observation from agent ()
        pdata_M_Doa = opt.last_sampbatch.obs.stacked # np.concatenate([opt.last_sampbatch.obs.stacked, opt.last_sampbatch.a.stacked], axis=1)

        # Plot reward
        _, ax = plt.subplots()
        idx1, idx2 = 0,1
        range1 = (min(exdata_N_Doa[:,idx1].min(), pdata_M_Doa[:,idx1].min()), max(exdata_N_Doa[:,idx1].max(), pdata_M_Doa[:,idx1].max()))
        range2 = (min(exdata_N_Doa[:,idx2].min(), pdata_M_Doa[:,idx2].min()), max(exdata_N_Doa[:,idx2].max(), pdata_M_Doa[:,idx2].max()))
        reward.plot(ax, idx1, idx2, range1, range2, n=100)
        
        # Plot policy samples
        ax.scatter(pdata_M_Doa[:,idx1], pdata_M_Doa[:,idx2], color='red',  alpha=0.5, s=1, label='apprentice')
        # Plot expert data
        ax.scatter(exdata_N_Doa[:,idx1], exdata_N_Doa[:,idx2], color='blue', alpha=0.5, s=1, label='expert')



        ax.legend()
        plt.show()

## 5. Evaluation : 학습된 Policy 실행시켜보기

In [0]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()
import os
os.environ["DISPLAY"] = ":" + str(display.display) + "." + str(display.screen)
import gym

returns = []
lengths = []
images = []

env = gym.make(env_name)
n = 10

for i_traj in xrange(n):
    obs = env.reset()    
    totalr = 0.
    l = 0
    done = False
    imgarr = []
    while not done:
        a = policy.sample_actions(obs[None], True)[0][0,0]        
        obs, r, done, _ = env.step(a)
        img = env.render(mode='rgb_array')
        imgarr.append(img)
        totalr += r
        l += 1
        if l>=200:
          break;
          
    returns.append(totalr)
    lengths.append(l)
    images.append(imgarr)
    print("-- Episode : %2d/%d | Return : %4.3f | Length : %4d "%(i_traj+1, n, totalr, l))
    
print("**** Return : Avg=%.3f , Std=%.3f"%(np.array(returns).mean(), np.array(returns).std()))

In [0]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from matplotlib import animation
from JSAnimation import IPython_display
from IPython.display import display, HTML

def plot_movie_mp4(image_array):
    dpi = 72.0
    xpixels, ypixels = image_array[0].shape[0], image_array[0].shape[1]
    fig = plt.figure(figsize=(ypixels/dpi, xpixels/dpi), dpi=dpi)
    im = plt.figimage(image_array[0])

    def animate(i):
        im.set_array(image_array[i])
        return (im,)

    anim = animation.FuncAnimation(fig, animate, frames=len(image_array))
    display(HTML(anim.to_html5_video()))

def plot_movie_js(image_array):
    dpi = 10.0
    xpixels, ypixels = image_array[0].shape[0], image_array[0].shape[1]
    fig = plt.figure(figsize=(ypixels/(dpi), xpixels/(dpi)), dpi=dpi)
    im = plt.figimage(image_array[0])

    def animate(i):
        im.set_array(image_array[i])
        return (im,)
    
    anim = animation.FuncAnimation(fig, animate, frames=len(image_array))
    display(IPython_display.display_animation(anim))
    
% matplotlib inline
plot_movie_js(images[7])
# plot_movie_mp4(images[6])

## 실습과제

1. 현재 실습 자료는 `CartPole-v0` 환경을 학습하도록 되어 있습니다. 먼저 `CartPole` Task가 잘 학습되는지 확인해보세요.

2. `MountainCar-v0` 환경을 쓰는 코드를 참고하여 학습하도록 바꿔보세요.

> **Hint** : `CartPole-v0`의 Expert Trajectory는 `expert_trajs/trajs_cartpole.h5`에 있습니다.

3. 학습이 잘 안된다면, 그 이유는 무엇일지 생각해보고 학습이 잘 되도록 하이퍼파라메터와 코드를 수정해봅시다.

> **Hint** : `CartPole` Task는 가능한 한 Episode가 끝나지 않도록 오래 유지해야하는 Task고, `MountainCar` Task는 가능한 한 Episode를 빨리 끝내야 하는 Task라는 점에서 차이가 있습니다.
